# Network Snapshot Query Demo
## Select a snapshot, load it into Neo4j, then ask questions

This notebook demonstrates:
1. List available snapshots
2. Select and load a snapshot into Neo4j
3. Query the network using natural language

In [1]:
# Setup
import sys
from pathlib import Path
sys.path.append(str(Path().absolute()))

from agents import SnapshotManager, NetworkQueryAgent
import json

c:\Matin\codes\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Matin\codes\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


## Step 1: List Available Snapshots

In [2]:
# Initialize snapshot manager
snapshot_mgr = SnapshotManager()

# Display all available snapshots
snapshots = snapshot_mgr.display_snapshots()


AVAILABLE NETWORK SNAPSHOTS
#    Timestamp            Devices    Size       Filename                      
--------------------------------------------------------------------------------
1    2026-01-04 14:48:23  4          24.7 KB    network_2026-01-04T14-48-23.641220.json
2    2026-01-04 14:41:36  6          43.5 KB    network_2026-01-04T14-41-36.944110.json
3    2026-01-04 14:15:30  6          44.1 KB    network_2026-01-04T14-15-30.785182.json


## Step 2: Select and Load a Snapshot

Choose a snapshot by index number from the list above.

In [3]:
# Check what's currently loaded (if anything)
current = snapshot_mgr.get_current_snapshot()
if current:
    print(f"Currently loaded: {current}")
else:
    print("No snapshot currently loaded")

Currently loaded: 2026-01-04T14:41:36.944110


In [9]:
# CHOOSE SNAPSHOT INDEX HERE (change the number)
SNAPSHOT_INDEX = 1  # Choose from the list above

# Clear existing data and load new snapshot
snapshot_mgr.clear_neo4j()
result = snapshot_mgr.load_snapshot(snapshot_index=SNAPSHOT_INDEX)

print(f"\nLoaded snapshot: {result['snapshot_id']}")
print(f"Total devices: {result['devices']}")
print(f"Total interfaces: {result['interfaces']}")
print(f"CDP connections: {result['cdp_connections']}")
print(f"OSPF connections: {result['ospf_connections']}")


[WARNING] Clearing all data from Neo4j...
[OK] Neo4j database cleared

[LOADING] network_2026-01-04T14-48-23.641220.json

FEEDING SNAPSHOT TO NEO4J
Snapshot ID: 2026-01-04T14:48:23.641220
Devices: 4

[PHASE 0] Creating Snapshot Node
[OK] Snapshot node created

[PHASE 1] Creating Device Nodes
[OK] Created 4 devices

[PHASE 2] Creating Interfaces
[OK] Created 57 interfaces

[PHASE 3] Storing Switch Extra Data
[OK] Extra data stored

[PHASE 4] Building Lookups
[OK] Indexed 57 interfaces, 4 devices

[PHASE 5] Creating CDP Connections
[OK] Created 1 CDP connections

[PHASE 6] Creating OSPF Connections
[OK] Created 0 OSPF connections

[SUCCESS] SNAPSHOT LOADED INTO NEO4J
  - Devices: 4
  - Interfaces: 57
  - CDP connections: 1
  - OSPF connections: 0

[OK] Active snapshot set to: 2026-01-04T14:48:23.641220

Loaded snapshot: 2026-01-04T14:48:23.641220
Total devices: 4
Total interfaces: 57
CDP connections: 1
OSPF connections: 0


## Step 3: Initialize Query Agent

In [5]:
# Initialize network query agent
agent = NetworkQueryAgent()

print("Query agent ready! You can now ask questions about the network.")

Query agent ready! You can now ask questions about the network.


## Step 4: Ask Questions About the Network

Try different questions:
- "Show me all devices"
- "What interfaces are down?"
- "Show neighbors of CORE-SW1"
- "Path between EDGE-R1 and ACC-SW2"
- "Show topology"
- "Count interfaces"

In [10]:
# Ask a question
question = "Show me all devices"

result = agent.ask(question)

# Display results
print(f"Question: {question}")
print(f"\nTemplate: {result['template']}")
print(f"Cypher: {result['cypher']}")
print(f"\nFound {result['count']} results:\n")

for i, record in enumerate(result['results'], 1):
    print(f"{i}. {record}")

Question: Show me all devices

Template: list_devices
Cypher: MATCH (d:Device)
RETURN d.hostname AS host, d.type AS type, d.ip_address AS ip
ORDER BY host

Found 4 results:

1. {'host': 'ACC-SW1', 'type': 'switch', 'ip': '10.10.10.4'}
2. {'host': 'ACC-SW2', 'type': 'switch', 'ip': '10.10.10.5'}
3. {'host': 'EDGE-R1', 'type': 'router', 'ip': '10.10.10.1'}
4. {'host': 'MANAGEMENT', 'type': 'switch', 'ip': '10.10.10.10'}


In [11]:
# Try another question
question = "What interfaces are down?"

result = agent.ask(question)

print(f"Question: {question}")
print(f"\nTemplate: {result['template']}")
print(f"\nFound {result['count']} results:\n")

for i, record in enumerate(result['results'], 1):
    print(f"{i}. {record}")

Question: What interfaces are down?

Template: find_down_interfaces

Found 44 results:

1. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet0/0', 'status': 'down', 'protocol': 'down'}
2. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet0/3', 'status': 'administratively down', 'protocol': 'down'}
3. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet1/0', 'status': 'down', 'protocol': 'down'}
4. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet1/1', 'status': 'down', 'protocol': 'down'}
5. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet1/2', 'status': 'down', 'protocol': 'down'}
6. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet1/3', 'status': 'down', 'protocol': 'down'}
7. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet2/0', 'status': 'down', 'protocol': 'down'}
8. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet2/1', 'status': 'down', 'protocol': 'down'}
9. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet2/2', 'status': 'down', 'protocol': 'down'}
10. {'host': 'ACC-SW1', 'iface': 'GigabitEthernet2/3', 'status': 'd

In [12]:
# Path query example
question = "Show path between EDGE-R1 and ACC-SW1"

result = agent.ask(question)

print(f"Question: {question}")
print(f"\nTemplate: {result['template']}")
print(f"\nFound {result['count']} path(s):\n")

for i, record in enumerate(result['results'], 1):
    print(f"{i}. {record}")

Question: Show path between EDGE-R1 and ACC-SW1

Template: show_shortest_path

Found 0 path(s):



In [ ]:
# Custom question - try your own!
question = "YOUR QUESTION HERE"

result = agent.ask(question)

print(f"Question: {question}")
print(f"\nReasoning:\n{result['reasoning']}")
print(f"\nTemplate: {result['template']}")
print(f"Cypher: {result['cypher']}")
print(f"\nFound {result['count']} results:\n")

for i, record in enumerate(result['results'], 1):
    print(f"{i}. {record}")

## Step 5: Load a Different Snapshot

You can load a different snapshot at any time

In [8]:
# Display snapshots again
snapshots = snapshot_mgr.display_snapshots()

# Choose a different snapshot
SNAPSHOT_INDEX = 1  # Change this

# Load it
snapshot_mgr.clear_neo4j()
result = snapshot_mgr.load_snapshot(snapshot_index=SNAPSHOT_INDEX)

print(f"\nLoaded new snapshot: {result['snapshot_id']}")
print("You can now ask questions about this snapshot!")


AVAILABLE NETWORK SNAPSHOTS
#    Timestamp            Devices    Size       Filename                      
--------------------------------------------------------------------------------
1    2026-01-04 14:48:23  4          24.7 KB    network_2026-01-04T14-48-23.641220.json
2    2026-01-04 14:41:36  6          43.5 KB    network_2026-01-04T14-41-36.944110.json
3    2026-01-04 14:15:30  6          44.1 KB    network_2026-01-04T14-15-30.785182.json

[WARNING] Clearing all data from Neo4j...
[OK] Neo4j database cleared

[LOADING] network_2026-01-04T14-48-23.641220.json

FEEDING SNAPSHOT TO NEO4J
Snapshot ID: 2026-01-04T14:48:23.641220
Devices: 4

[PHASE 0] Creating Snapshot Node
[OK] Snapshot node created

[PHASE 1] Creating Device Nodes
[OK] Created 4 devices

[PHASE 2] Creating Interfaces
[OK] Created 57 interfaces

[PHASE 3] Storing Switch Extra Data
[OK] Extra data stored

[PHASE 4] Building Lookups
[OK] Indexed 57 interfaces, 4 devices

[PHASE 5] Creating CDP Connections
[OK] Creat

## Cleanup

In [ ]:
# Close connections when done
agent.close()
snapshot_mgr.close()

print("Connections closed")